# Z peak example
This code is designed to run a self-contained simple analysis of CMS data in a columnar format.
No part of this should be construed as a finalzed publication-quality analysis: it is incomplete, many of the corrections are simplistic and/or out of date, and no systematic effects have been considered.  It is a technical demonstration only.
The data must be downloaded locally, via `download.sh` script in this directory.
(A CMS grid certificate will be necessary)

The core of the analysis is found in the cell with the heading `#__worker_class__`.
The structure of the Worker class is driven by the needs of the 'striped' data delivery system, although here we only use uproot to do data delivery, as encapsulated by the `UprootJob` class.

In [ ]:
from __future__ import division, print_function
from collections import OrderedDict
import cloudpickle
import zlib
import json
import gzip

import numpy as np
%matplotlib nbagg
import matplotlib.pyplot as plt

import uproot

import fnal_column_analysis_tools.lumi_tools as lumi_tools
import fnal_column_analysis_tools.lookup_tools as lookup_tools
import fnal_column_analysis_tools.hist as hist

In [ ]:
# Mapping dataset -> files
with open("metadata/datadef_nano.json") as fin:
    datadef = json.load(fin)

In [ ]:
# Grab luminosity values and valid data masks
# wget https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/Collisions17/13TeV/Final/Cert_294927-306462_13TeV_PromptReco_Collisions17_JSON.txt
# brilcalc lumi -c /cvmfs/cms.cern.ch/SITECONF/local/JobConfig/site-local-config.xml -b "STABLE BEAMS" --normtag=/cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u /pb --byls --output-style csv -i Cert_294927-306462_13TeV_PromptReco_Collisions17_JSON.txt > lumi2017.csv
lumidata = lumi_tools.LumiData("metadata/lumi2017.csv.gz")
lumimask = lumi_tools.LumiMask("metadata/Cert_294927-306462_13TeV_PromptReco_Collisions17_JSON.txt")
lumimask_pkl = zlib.compress(cloudpickle.dumps(lumimask))

# Import a bunch of correction histograms
weightsext = lookup_tools.extractor()
correctionDescriptions = open("metadata/corrections.txt").readlines()
weightsext.add_weight_sets(correctionDescriptions)
weightsext.finalize()
# print(list(weightsext._names.keys()))
weights_eval = weightsext.make_evaluator()
#let's pickle and zip it
weval_pickle = zlib.compress(cloudpickle.dumps(weights_eval))

In [ ]:
# Define the histograms

hists = OrderedDict()

dataset = hist.Cat("dataset", "DAS name")
channel = hist.Cat("channel", "dilepton flavor")

# underflow = negative weight sum, overflow = positive weight sum
hists['genw'] = hist.Hist("Events", dataset, hist.Bin("genw", "Gen weight", [0.]))

hists['lepton_pt'] = hist.Hist("Events", dataset, channel, 
                               hist.Bin("lep0_pt", "Leading lepton $p_{T}$ [GeV]", 50, 0, 500),
                               hist.Bin("lep1_pt", "Trailing lepton $p_{T}$ [GeV]", 50, 0, 500),
                              )
hists['zMass'] = hist.Hist("Events", dataset, channel,
                           hist.Bin("mass", "$m_{\ell\ell}$ [GeV]", 120, 0, 120),
                          )

hists['profile'] = hist.Hist("RowGroups", 
                    hist.Cat("op", "Operation", sorting='placement'),
                    hist.Bin("dt", "$\Delta t$ [$\mu s$]", 100, 0, 10),
                   )

hists['trigger'] = hist.Hist("Counts", dataset, hist.Cat("trigger", "Trigger name"))

# These aren't actually histograms but have the same reducable property with +=
hists['lumis_ee'] = lumi_tools.LumiList()
hists['lumis_mm'] = lumi_tools.LumiList()

In [ ]:
# This defines the mapping from dataset to 'process', or element of the stack plot
# as well as the cross sections (for later use)
process = hist.Cat("process", "Process", sorting='placement')
process_map = OrderedDict()
process_map["TTbar"] = "TTJets*"
process_map["ZJets"] = "DYJetsToLL*"
process_map["Data"] = ["DoubleEG", "DoubleMuon"]

import re
mc = re.compile("^(?!Data)")
data = "Data"

# xs = {k:v['xs'] for k,v in datadef.items() if v['xs'] > 0.}
xs = {
    "DYJetsToLL_M-10to50_TuneCP5_13TeV-madgraphMLM-pythia8": 18610.,
    "DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8": 1921.8*3,
    "TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8": 87.3,
}

def processmap(hist):
    return hist.group(process, "dataset", process_map)

In [ ]:
# This is to extract the pileup distribution in simulation
puBins = np.arange(101)

mcPu = {}
for dataset, info in datadef.items():
    if dataset in ["DoubleEG", "DoubleMuon"]:
        continue
    flist = info['files']
    puHist = np.zeros(puBins.size-1)
    for f in flist:
        pu = uproot.open(f)['Events']['Pileup_nTrueInt'].array()
        np.add.at(puHist, np.clip(np.searchsorted(puBins, pu, side='right'), 0, puHist.size-1), 1.)
    mcPu[dataset] = puHist


In [ ]:
# This is to extract the pileup distribution in data

# https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/Collisions17/13TeV/PileUp/pileup_latest.txt
with gzip.open("metadata/pileup_2017.json.gz") as fin:
    dataPuIn = json.load(fin)

dataPu = []
for run, info in dataPuIn.items():
    for lumiinfo in info:
        if lumimask(np.array(int(run)), np.array(lumiinfo[0])):
            dataPu.append([int(run)]+lumiinfo)

dataPuIn = None
dataPu = np.array(dataPu)

# Run complete job to get lumilist, then rerun this cell restrict
# pileup reweighting to the lumisections corresponding to the data we actually analyze
# (which is a tiny portion of Run2017B)
if hists['lumis_ee'].array.size > 0:
    didx = dataPu[:,:2].astype('u4').view([('run', 'u4'), ('lumi', 'u4')])
    rlidx = hists['lumis_ee'].array.astype('u4').view([('run', 'u4'), ('lumi', 'u4')])
    dataPu = dataPu[np.isin(didx, rlidx)[:,0]]

# Compute data pileup distribution from per-lumisection
# instantaneous luminosity distribution (parameterized by mean and width)
from scipy.special import erf
mBxsec = 69200.
intLumi = dataPu[:,2]
meanPu = dataPu[:,4]*mBxsec
stdPu = dataPu[:,3]*mBxsec
dataPuDist = np.sum(
    np.diff(
        erf(np.subtract.outer(puBins, meanPu)/np.maximum(stdPu,1e-6)/np.sqrt(2)),
        axis=0
    )*intLumi,
    axis=1
)

In [ ]:
# Draw the pileup distributions to check
fig, ax = plt.subplots()
ax.step(x=puBins, y=np.r_[dataPuDist/dataPuDist.sum(), 0], label='Data PU')
mcs = mcPu['DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8']
ax.step(x=puBins, y=np.r_[mcs/mcs.sum(), 0], label='MC PU (DYJetsToLL)')
ax.legend()

In [ ]:
# Compute correction factor
puCorr = {}
for k in mcPu:
    corr = (dataPuDist / np.maximum(mcPu[k], 1)) / (dataPuDist.sum() / mcPu[k].sum())
    puCorr[k] = lookup_tools.dense_lookup.dense_lookup(corr, puBins)
puCorr_pkl = zlib.compress(cloudpickle.dumps(puCorr))

In [ ]:
# Save correction factor (for use in the ROOT version :)
import os
if os.path.exists("correction_files/puReweight0p38fb.root"):
    os.remove("correction_files/puReweight0p38fb.root")
fout = uproot.create("correction_files/puReweight0p38fb.root")
for k,v in puCorr.items():
    h = hist.Hist("Ratio", hist.Bin("pu","N pileup", v._axes))
    h._sumw = {(): np.r_[0, v._values, 0, 0]}
    fout[k] = hist.export1d(h, axis="pu")

fout.close()

In [ ]:
#__worker_class__

# This receives the data as numpy arrays attached to the 'events' object passed to Worker.run()
# It fills histograms and sends them back to the job collector.  In a distributed system, this
# would be a non-trivial operation, but here it is just a facade.

import numpy as np
import cloudpickle
import copy
import zlib
import time
from collections import OrderedDict
from fnal_column_analysis_tools.striped import ColumnGroup, PhysicalColumnGroup, jaggedFromColumnGroup
from fnal_column_analysis_tools.lumi_tools import LumiList

class Worker(object):
    def __init__(self):
        self.Columns = []
        # https://cms-nanoaod-integration.web.cern.ch/integration/master-102X/mc80X_doc.html
        self.Columns.extend(["run", "luminosityBlock", "LHEWeight_originalXWGTUP", "Pileup_nTrueInt"])
        
        self.electron_cols = ["pt", "eta", "phi", "mass", "cutBased", "pdgId", "pfRelIso03_all"]
        self.Columns.extend("Electron."+col for col in self.electron_cols)

        self.muon_cols = ["pt", "eta", "phi", "mass", "tightId", "pdgId", "pfRelIso04_all"]
        self.Columns.extend("Muon."+col for col in self.muon_cols)

        # Define here dataset priority for overlap removal
        self.triggers = OrderedDict()
        self.triggers["DoubleEG"] = [
                "HLT_Ele32_WPTight_Gsf_L1DoubleEG",
                "HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ",
            ]
        self.triggers["DoubleMuon"] = [
                "HLT_IsoMu24",
                "HLT_IsoMu27",
                "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ",
            ]
        for triggers in self.triggers.values(): self.Columns.extend(triggers)
        
        self.weights_eval = None
        self.lumimask = None
        self.puCorr = None
        
    def run(self, events, job):
        times = OrderedDict()
        times['start'] = time.time()
        
        if self.weights_eval is None:
            self.weights_eval = cloudpickle.loads(zlib.decompress(job["weights_eval"]))
        if self.lumimask is None:
            self.lumimask = cloudpickle.loads(zlib.decompress(job["lumimask"]))
        if self.puCorr is None:
            self.puCorr = cloudpickle.loads(zlib.decompress(job["puCorr"]))


        dataset = job['dataset']
        hists = job['hists']

        isRealData = False
        good_trigger = np.zeros(events.nevents, dtype=bool)
        if dataset in self.triggers:
            isRealData = True
            # Real data, prevent overlaps
            overlap_veto = np.zeros_like(good_trigger)
            for ds, tlist in self.triggers.items():
                if ds != dataset:
                    for trigger in tlist:
                        overlap_veto |= getattr(events, trigger)
                        hists['trigger'].fill(dataset=dataset, trigger=trigger, weight=getattr(events, trigger).sum())
                else:
                    for trigger in tlist:
                        good_trigger |= getattr(events, trigger)
                        hists['trigger'].fill(dataset=dataset, trigger=trigger, weight=getattr(events, trigger).sum())
                    break
            good_trigger &= ~overlap_veto
            # apply 'golden json' certified lumiblocks
            good_trigger &= lumimask(events.run, events.luminosityBlock)
            # record lumis processed
            #  in uproot jobs, full lumi completion guaranteed by full-file processing
            #  in striped jobs, partial lumi completion is not traceable at present
            if dataset == "DoubleEG":
                hists['lumis_ee'] += LumiList(events.run, events.luminosityBlock)
            elif dataset == "DoubleMuon":
                hists['lumis_mm'] += LumiList(events.run, events.luminosityBlock)
        else:
            # MC, OR all
            for tlist in self.triggers.values():
                for trigger in tlist:
                    good_trigger |= getattr(events, trigger)
                    hists['trigger'].fill(dataset=dataset, trigger=trigger, weight=getattr(events, trigger).sum())
            genw = np.sign(events.LHEWeight_originalXWGTUP)
            puw = self.puCorr[dataset](events.Pileup_nTrueInt)
        
        times['lumimask, trigger'] = time.time()
       
        electronCols = PhysicalColumnGroup(events, "Electron", **{k:k for k in self.electron_cols})
        electrons = jaggedFromColumnGroup(electronCols)  
        electrons['weight'] = weights_eval["eleScaleFactor_TightId_POG"](electrons.p4.eta, electrons.p4.pt)

        muonCols = PhysicalColumnGroup(events, "Muon", **{k:k for k in self.muon_cols})
        muons = jaggedFromColumnGroup(muonCols)
        muons['weight'] = weights_eval["muScaleFactor_TightId_Iso"](np.abs(muons.p4.eta), muons.p4.pt)
                
        ele = electrons[(electrons.p4.pt > 20) &
                                           (np.abs(electrons.p4.eta) < 2.5) &
                                           (electrons.cutBased >= 4)]
        
        mu = muons[(muons.p4.pt > 20) &
                                   (np.abs(muons.p4.eta) < 2.4) &
                                   (muons.tightId > 0)]
        
        times['good leptons'] = time.time()

        ee = ele.distincts()
        mm = mu.distincts()
        em = ele.cross(mu)
        
        dileptons = {}
        dileptons['ee'] = ee[(ee.i0.pdgId*ee.i1.pdgId == -11*11) & (ee.i0.p4.pt > 25)]
        dileptons['mm'] = mm[(mm.i0.pdgId*mm.i1.pdgId == -13*13)]
        dileptons['em'] = em[(em.i0.pdgId*em.i1.pdgId == -11*13)]
        
        times['good pairs'] = time.time()
        
        channels = {}
        channels['ee'] = good_trigger & (ee.counts == 1) & (mu.counts == 0)
        channels['mm'] = good_trigger & (mm.counts == 1) & (ele.counts == 0)
        channels['em'] = good_trigger & (em.counts == 1) & (ele.counts == 1) & (mu.counts == 1)
        
        times['channels'] = time.time()
        
        if not isRealData:
            hists['genw'].fill(dataset=dataset, genw=genw)

        dupe = np.zeros(events.nevents, dtype=bool)
        tot = 0
        for channel, cut in channels.items():
            zcands = dileptons[channel][cut]
            dupe |= cut
            tot += cut.sum()
            weight = np.array(1.)
            if not isRealData:
                weight = zcands.i0['weight'] * zcands.i1['weight'] * genw[cut] * puw[cut]
            hists['zMass'].fill(dataset=dataset, channel=channel,
                                mass=zcands.p4.mass.flatten(),
                                weight=weight.flatten(),
                               )
            # only fill for m > 50
            weight = weight.flatten() * (zcands.p4.mass > 50.).flatten()
            hists['lepton_pt'].fill(dataset=dataset, channel=channel, 
                                    lep0_pt=zcands.i0.pt.flatten(),
                                    lep1_pt=zcands.i0.pt.flatten(),
                                    weight=weight,
                                   )
      
        if dupe.sum() != tot:
            raise Exception("Double-counting events!")
            
        times['plots'] = time.time()
        
        #profiling info
        t = list(times.values())
        for i, name in enumerate(times):
            if i==0: continue
            dt = t[i] - t[i-1]
            hists['profile'].fill(op=name, dt=1e6*(dt/len(events.Muon.count)))
        hists['profile'].fill(op="total", dt=1e6*(t[-1]-t[0])/events.nevents)
        
        job.send(hists=hists)

In [ ]:
# This sets up and displays real-time-updateable set of plots
from fnal_column_analysis_tools.hist import plot
from fnal_column_analysis_tools.striped import HistCollectorCallback

fill_opts = {'edgecolor': (0,0,0,0.3), 'alpha': 0.8}
error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'none', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_error_opts = {'linestyle':'none', 'marker': '.', 'markersize': 10., 'color':'k', 'elinewidth': 1, 'emarker': '_', 'drawstyle': 'default'}

class PlotDrawer(object):    
    def __call__(self):
        figs = []
        figs.append(self.drawmass_ee())
        figs.append(self.drawprofile())
        return figs
    
    def drawmass_ee(self):
        figin = getattr(self, 'massfig', None)
        lumi = lumidata.get_lumi(hists['lumis_mm'])
        scale = {k[0]: lumi*xs[k[0]]/(v[1]-v[0]) for k,v in hists['genw'].values(overflow='all').items() if k[0] in xs}
        hscaled = hists['zMass'].project("channel", "mm")
        hscaled.scale(scale, axis="dataset")
        hproc = processmap(hscaled)
        fig, ax, _ = plot.plot1d(hproc[mc], overlay='process', ax=figin, stack=True, fill_opts=fill_opts, error_opts=error_opts)
        plot.plot1d(hproc[data], overlay='process', ax=ax, clear=False, error_opts=data_error_opts)
        ax.text(1., 1., r"%.2f fb$^{-1}$ (13 TeV)" % (lumi*1e-3), fontsize=16, horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes)
        ax.set_yscale('log')
        ax.autoscale(axis='y', tight=False)
        ax.set_ylim(1, None)
        self.massfig = ax
        return fig

    def drawprofile(self):
        axin = getattr(self, 'profileaxis', None)
        fig, axis, _ = plot.plot1d(hists['profile'], overlay='op', ax=axin, line_opts={})
        axis.legend(title=hists['profile'].axis('op').label)
        self.profileaxis = axis
        return fig


draw = PlotDrawer()
callback = HistCollectorCallback(hists, draw, update=1e6)
# This cell must stay separated from the update loop
# https://github.com/matplotlib/jupyter-matplotlib/issues/17
figs = draw()

In [ ]:
# This runs the worker over the data we defined earlier
from fnal_column_analysis_tools.striped import UprootJob

for h in hists.values(): h.clear()
    
from pyinstrument import Profiler
profiler = Profiler()

t0 = time.time()
profiler.start()

threads = {}
events = 0
for dataset, info in datadef.items():
    flist = info['files']
    job = UprootJob(
        dataset=dataset,
        filelist=flist,
        treename="Events",
        worker_class=Worker,
        user_callback=callback,
        user_params={
                "weights_eval": weval_pickle,
                "lumimask": lumimask_pkl,
                "puCorr": puCorr_pkl,
                "dataset": dataset,
                "hists": hists,
            },
    )
    threads[dataset] = job.run(workers=0, progress=False, threaded=False)
    events += job.EventsProcessed

t1 = time.time()
profiler.stop()
print(profiler.output_text(unicode=True, color=True))

In [ ]:
(t1-t0)*1e6/events

In [ ]:
figin = None
lumi = lumidata.get_lumi(hists['lumis_mm'])
scale = {k[0]: lumi*xs[k[0]]/(v[1]-v[0]) for k,v in hists['genw'].values(overflow='all').items() if k[0] in xs}
hscaled = hists['lepton_pt'].project("channel", "mm").sum('lep1_pt')
hscaled.scale(scale, axis="dataset")
hproc = processmap(hscaled)
fig, ax, _ = plot.plot1d(hproc[mc], overlay='process', ax=figin, stack=True, fill_opts=fill_opts, error_opts=error_opts)
plot.plot1d(hproc[data], overlay='process', ax=ax, clear=False, error_opts=data_error_opts)
ax.text(1., 1., r"%.2f fb$^{-1}$ (13 TeV)" % (lumi*1e-3), fontsize=16, horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes)
ax.set_yscale('log')
ax.set_ylim(1, 1e7)


In [ ]:
hists['genw'].values(overflow='all')

In [ ]:
with open("profile.html", "w") as fout:
    fout.write(profiler.output_html())

In [ ]:
import gzip, pickle
with gzip.open("hists.pkl.gz", "wb") as fout:
    pickle.dump(hists, fout)

In [ ]:
# This cell is currently not functional

from striped.job import SinglePointStripedSession as Session
job_server = ("ifdb02.fnal.gov", 8766)
session = Session(job_server)

nevents_total = 0
t1 = time.time()
for dataset in datasets:
    job = session.createJob(dataset, 
            fraction=0.1,
            user_callback=callback,
            user_params = {
                "weights_eval": weval_pickle,
                "dataset": dataset,
                "hists": hists,
                "profile": profile,
            },
    )
    job.run()
    runtime = job.TFinish - job.TStart
    nevents = job.EventsProcessed
    nevents_total += nevents
    print("%-70s %7.3f M events, %7.3f M events/sec" % (dataset[:70], float(nevents)/1e6, nevents/runtime/1000000))

t2 = time.time()
print("Total events processed: %d in %.1f seconds -> %.6f million events/second" %(nevents_total, t2-t1, nevents_total/(t2-t1)/1000000))